In [25]:
import pyomo.environ as pe
import pyomo.opt as po

### Problema Data base

Cristina Hernández, Macarena Vargas, Guillermo Ruiz, Beatriz Jiménez. 3ºIMAT B 

In [26]:
model = pe.ConcreteModel()

#### SETS
$f$: files {A,B,C,D,E,F,G}

$t$: folder {F1,F2,F3}

$s$: search {S1,S2,S3,S4}


In [27]:
files = ['A','B','C','D','E','F','G']
folders = ['F1','F2','F3']
searches = ['S1','S2','S3','S4']

search_files = {
    'S1': ['A','D'],        
    'S2': ['C','E','F'],    
    'S3': ['B','D','G'],    
    'S4': ['A','B','G'],   
}

model.f = pe.Set(initialize=files, ordered=True)     
model.t = pe.Set(initialize=folders, ordered=True)      
model.s = pe.Set(initialize=searches, ordered=True)    


$P_s$: probability of search s [%]

$N_s$: number of files in search s

$T$: (unknown) time of one folder search [s]

$e_fs$: existing file f in search s (binary)

In [28]:
P_dict = {'S1': 35, 'S2': 25, 'S3': 20, 'S4': 20}

N_dict = {s: len(search_files[s]) for s in searches}

# iniciamos e_f_s como 1 si el archivo f existe en la busqueda s y 0 en el caso contrario 
e_dict = {(f, s): 0 for f in files for s in searches}
for s in searches:
    for f in search_files[s]:
        e_dict[(f, s)] = 1

# incializamos el tiempo de busqueda en 1 por ejemplo (da igual)
T_value = 1.0

model.P = pe.Param(model.s, initialize=P_dict)                  
model.N = pe.Param(model.s, initialize=N_dict)                    
model.e = pe.Param(model.f, model.s, initialize=e_dict, default=0, within=pe.Binary) 
model.T = pe.Param(initialize=T_value)       

#### Variables
$x_ft$: distribution of file f to folder t {0,1}

$y_ts$: consult folder t for search s {0,1}

$u_s$: # of folders used in search s {1,2,3}

In [29]:
model.x = pe.Var(model.f, model.t, within=pe.Binary)
model.y = pe.Var(model.t, model.s, within=pe.Binary)
model.u = pe.Var(model.s, within=pe.NonNegativeIntegers, bounds=(1, 3))

#### Objective Function
min $\sum_{s}(P_{s} * T * (0.75+0.25*u_s) )$

In [30]:
def expected_time(m):
    return sum(m.P[s] * m.T * (0.75 + 0.25*m.u[s]) for s in m.s)

model.Obj = pe.Objective(rule=expected_time, sense=pe.minimize)

#### Constrains

In [31]:
# cada archivo exactamente en una carpeta
model.un_archivo_por_carpeta = pe.ConstraintList()
for f in model.f:
    model.un_archivo_por_carpeta.add(sum(model.x[f, t] for t in model.t) == 1)

# cada carpeta como mucho 4 archivos
model.max_capacidad = pe.ConstraintList()
for t in model.t:
    model.max_capacidad.add(sum(model.x[f, t] for f in model.f) <= 4)

In [32]:
# ficheros del folder t usados en s
model.ficheros_t_en_s = pe.ConstraintList()
for t in model.t:
    for s in model.s:
        #numero de ficheros de s en t
        num_ficheros_s_en_t = sum(model.e[f, s] * model.x[f, t] for f in model.f)
        
        model.ficheros_t_en_s.add(num_ficheros_s_en_t <= model.N[s] * model.y[t, s])

# numero de carpetas usadas por la búsqueda s
model.num_carpetas_busqueda = pe.ConstraintList()
for s in model.s:
    model.num_carpetas_busqueda.add(sum(model.y[t, s] for t in model.t) <= model.u[s])

In [33]:

# la carpeta que contiene A no puede contener F
model.no_A_F = pe.ConstraintList()
for t in model.t:
    model.no_A_F.add(model.x['A', t] + model.x['F', t] <= 1)

    
# si una carpeta contiene A y B, entonces debe contener D y E
# añadimos la variable delta que solo usamos para esta constrain
model.delta = pe.Var(model.t, within=pe.Binary)
model.si_AB_entonces_DE = pe.ConstraintList()
for t in model.t:
    model.si_AB_entonces_DE.add(model.x['A', t] + model.x['B', t] <= 1 + model.delta[t])
    model.si_AB_entonces_DE.add(model.x['D', t] + model.x['E', t] >= 2 * model.delta[t])

Resolvemos el problema 

In [34]:
solver = po.SolverFactory('gurobi')
resultado = solver.solve(model)

print ("estado:", resultado.solver.status)
print ("Condición de terminacion:", resultado.solver.termination_condition)

estado: ok
Condición de terminacion: optimal


In [35]:
print("archivos por carpeta")
for t in model.t:
    asignados = [f for f in model.f if pe.value(model.x[f, t]) > 0.5]
    print(f"{t}: {asignados}")

print("carpetas por busqueda")
for s in model.s:
    used = [t for t in model.t if pe.value(model.y[t, s]) > 0.5]
    print(f"  {s}: {used}   (u={int(round(pe.value(model.u[s])))})")

print("tiempo esperado:", pe.value(model.Obj))

archivos por carpeta
F1: ['A', 'D', 'G']
F2: ['B', 'C', 'E', 'F']
F3: []
carpetas por busqueda
  S1: ['F1']   (u=1)
  S2: ['F2']   (u=1)
  S3: ['F1', 'F2']   (u=2)
  S4: ['F1', 'F2']   (u=2)
tiempo esperado: 110.0
